# Objetivo

Esta ETL (Extract, Transform and Load) tem o objetivo principal de tratar os dados brutos da Olist com foco em criar tabelas que poderão ser utilizadas nos modelos de machine learning. 

Para isso, precisamos primeiro extrair, tratar e carregar e salvar as tabelas finais (fato) com as agregações e granularidade que precisamos.

Fonte das tabelas usadas neste notebook: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce.

# Bibliotecas

In [1]:
import os
import shutil
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt

/Users/joaonogueira/Documents/Denise/Fase 3/Projetos/tech_challenge_fase_03/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Carregando as tabelas

In [2]:
new_path = "../data/00_raw/"

# Se a pasta já existe e tem arquivos, apenas avisa
if os.path.exists(new_path) and len(os.listdir(new_path)) == 9:
    print(f"Já existem arquivos em '{new_path}'. Nenhum download necessário.")
else:
    # Remove a pasta de destino se já existir e tiver menos de 9 arquivos
    shutil.rmtree(new_path)
    # Faz o download do dataset
    path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
    print("Download concluído. Path:", path)
    # Move os arquivos baixados para a pasta de destino
    shutil.move(path, new_path)
    print(f"Arquivos movidos para '{new_path}'.")

Já existem arquivos em '../data/00_raw/'. Nenhum download necessário.


# Lendo as tabelas

In [3]:
# Vendedores
sellers = pd.read_csv(f"{new_path}olist_sellers_dataset.csv")

# Consumidores
customers = pd.read_csv(f"{new_path}olist_customers_dataset.csv")

# Geolocalização
geolocation = pd.read_csv(f"{new_path}olist_geolocation_dataset.csv")

# Produtos
products = pd.read_csv(f"{new_path}olist_products_dataset.csv")

# Categoria dos produtos
product_category_name_translation = pd.read_csv(f"{new_path}product_category_name_translation.csv")

# Produtos pedidos
order_items = pd.read_csv(f"{new_path}olist_order_items_dataset.csv")

# Pagamentos dos pedidos
order_payments = pd.read_csv(f"{new_path}olist_order_payments_dataset.csv")

# Status dos pedidos
orders = pd.read_csv(f"{new_path}olist_orders_dataset.csv")

# Reviews dos pedidos
order_reviews = pd.read_csv(f"{new_path}olist_order_reviews_dataset.csv")

# Tratamento dos dados

## Pedido

In [4]:
# Tabela fato de ordem de pedidos
valor_ordem_pedido = (
    orders
    .merge(order_items, on="order_id", how="inner") # deixando apenas pedidos que também estejam presentes na tabela de produtos pedidos
    .groupby(["order_id"])
    .agg(
        items_sold = ("product_id", "count"),
        unique_items_sold = ("product_id", "nunique"),
        price = ("price", "sum"),
        freight_value = ("freight_value", "sum"),
    )
    .reset_index()
)

valor_ordem_pedido["order_value"] = round(valor_ordem_pedido["price"] + valor_ordem_pedido["freight_value"],2)

# Deixando a tabela apenas com pedidos contidos na tabela de preço dos pedidos
df_fct_ordem_pedido = (
    orders
    .merge(valor_ordem_pedido, on="order_id", how="inner")
)

In [5]:
valor_ordem_pedido.head()

,order_id,items_sold,unique_items_sold,price,freight_value,order_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,1,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,1,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,1,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,1,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,1,199.90,18.14,218.04


In [6]:
df_fct_ordem_pedido.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,items_sold,unique_items_sold,price,freight_value,order_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,1,29.99,8.72,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,1,118.70,22.76,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,1,159.90,19.22,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,1,45.00,27.20,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,1,19.90,8.72,28.62


In [7]:
valor_ordem_pedido[valor_ordem_pedido["order_id"]=="0008288aa423d2a3f00fcb17cd7d8719"]

,order_id,items_sold,unique_items_sold,price,freight_value,order_value
13,0008288aa423d2a3f00fcb17cd7d8719,2,1,99.8,26.74,126.54


In [8]:
order_items[order_items["order_id"]=="0008288aa423d2a3f00fcb17cd7d8719"]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
13,0008288aa423d2a3f00fcb17cd7d8719,1,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.9,13.37
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.9,13.37


In [9]:
df_fct_ordem_pedido[df_fct_ordem_pedido["order_id"]=="0008288aa423d2a3f00fcb17cd7d8719"]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,items_sold,unique_items_sold,price,freight_value,order_value
53537,0008288aa423d2a3f00fcb17cd7d8719,2355af7c75e7c98b43a87b2a7f210dc5,delivered,2018-02-13 22:10:21,2018-02-15 03:55:52,2018-02-20 18:23:28,2018-02-26 13:55:22,2018-03-06 00:00:00,2,1,99.8,26.74,126.54


In [10]:
df_fct_ordem_pedido.info()
print(df_fct_ordem_pedido.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98666 entries, 0 to 98665
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       98666 non-null  object 
 1   customer_id                    98666 non-null  object 
 2   order_status                   98666 non-null  object 
 3   order_purchase_timestamp       98666 non-null  object 
 4   order_approved_at              98652 non-null  object 
 5   order_delivered_carrier_date   97657 non-null  object 
 6   order_delivered_customer_date  96476 non-null  object 
 7   order_estimated_delivery_date  98666 non-null  object 
 8   items_sold                     98666 non-null  int64  
 9   unique_items_sold              98666 non-null  int64  
 10  price                          98666 non-null  float64
 11  freight_value                  98666 non-null  float64
 12  order_value                    98666 non-null 

In [11]:
df_fct_ordem_pedido.describe(exclude='object')

,items_sold,unique_items_sold,price,freight_value,order_value
count,98666.000000,98666.000000,98666.000000,98666.000000,98666.000000
mean,1.141731,1.038098,137.754076,22.823562,160.577638
std,0.538452,0.226456,210.645145,21.650909,220.466087
min,1.000000,1.000000,0.850000,0.000000,9.590000
25%,1.000000,1.000000,45.900000,13.850000,61.980000
50%,1.000000,1.000000,86.900000,17.170000,105.290000
75%,1.000000,1.000000,149.900000,24.040000,176.870000
max,21.000000,8.000000,13440.000000,1794.960000,13664.080000


In [12]:
# Percentual de pedidos por status
order_status = df_fct_ordem_pedido.copy()
order_status['order_purchase_timestamp'] = pd.to_datetime(order_status['order_purchase_timestamp'])
order_status['year_month'] = order_status['order_purchase_timestamp'].dt.to_period('M')

order_status_agg = (
    order_status
    .groupby(["order_status"])
    .agg(count = ("order_id", "nunique"))
    .reset_index()
)
order_status_agg["proportion"] = round(order_status_agg["count"] / order_status_agg["count"].sum() * 100, 1)
order_status_agg = order_status_agg.sort_values(by="count", ascending=False)
order_status_agg

,order_status,count,proportion
2,delivered,96478,97.8
5,shipped,1106,1.1
1,canceled,461,0.5
3,invoiced,312,0.3
4,processing,301,0.3
6,unavailable,6,0.0
0,approved,2,0.0


### Salva os dados

In [13]:
df_fct_ordem_pedido.to_csv("../data/01_interim/tb_fct_ordem_pedido.csv", index=False)

## Categoria do pedido por vendedor

In [14]:
order_seller = (
    order_items
    .merge(products, on="product_id", how="left") #categorias de produtos vendidas mas nulas serão substituídas por "unknown"
    .merge(orders, on="order_id", how="left")
)

df_fct_ordem_vendedor_cat = (
    order_seller
    .groupby(["order_purchase_timestamp", "order_id", "order_status", "seller_id", "product_category_name"])
    .agg(
        items_sold = ("product_id", "count"),
        unique_items_sold = ("product_id", "nunique"),
        total_price = ("price", "sum"),
        avg_price = ("price", "mean"),
        min_price = ("price", "min"),
        max_price = ("price", "max"),
        median_price = ("price", "median"),
        std_price = ("price", "std"),
        total_freight_value = ("freight_value", "sum"),
        avg_freight_value = ("freight_value", "mean"),
        min_freight_value = ("freight_value", "min"),
        max_freight_value = ("freight_value", "max"),
        median_freight_value = ("freight_value", "median"),
        std_freight_value = ("freight_value", "std"),
    )
    .reset_index()
)

In [15]:
df_fct_ordem_vendedor_cat.head()

,order_purchase_timestamp,order_id,order_status,seller_id,product_category_name,items_sold,unique_items_sold,total_price,avg_price,min_price,max_price,median_price,std_price,total_freight_value,avg_freight_value,min_freight_value,max_freight_value,median_freight_value,std_freight_value
0,2016-09-04 21:15:19,2e7a8482f6fb09756ca50c10d7bfc047,shipped,1554a68530182680ad5c8b042c3ab563,moveis_decoracao,2,2,72.89,36.445,32.90,39.99,36.445,5.013387,63.34,31.67,31.67,31.67,31.67,0.0
1,2016-09-05 00:15:34,e5fa5a7210941f7d56d0208e4e071d35,canceled,a425f92c199eb576938df686728acd20,telefonia,1,1,59.50,59.500,59.50,59.50,59.500,NaN,15.56,15.56,15.56,15.56,15.56,NaN
2,2016-09-15 12:16:38,bfbd0f9bdef84302105ad712db648a6c,delivered,ecccfa2bb93b34a3bf033cc5d1dcdc69,beleza_saude,3,1,134.97,44.990,44.99,44.99,44.990,0.000000,8.49,2.83,2.83,2.83,2.83,0.0
3,2016-10-02 22:07:52,71303d7e93b399f5bcd537d124c0bcfa,canceled,25e6ffe976bd75618accfe16cefcbd0d,bebes,1,1,100.00,100.000,100.00,100.00,100.000,NaN,9.34,9.34,9.34,9.34,9.34,NaN
4,2016-10-03 09:44:50,3b697a20d9e427646d92567910af6d57,delivered,522620dcb18a6b31cd7bdf73665113a9,relogios_presentes,1,1,29.90,29.900,29.90,29.90,29.900,NaN,15.56,15.56,15.56,15.56,15.56,NaN


In [16]:
df_fct_ordem_vendedor_cat.info()
print(df_fct_ordem_vendedor_cat.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98762 entries, 0 to 98761
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   order_purchase_timestamp  98762 non-null  object 
 1   order_id                  98762 non-null  object 
 2   order_status              98762 non-null  object 
 3   seller_id                 98762 non-null  object 
 4   product_category_name     98762 non-null  object 
 5   items_sold                98762 non-null  int64  
 6   unique_items_sold         98762 non-null  int64  
 7   total_price               98762 non-null  float64
 8   avg_price                 98762 non-null  float64
 9   min_price                 98762 non-null  float64
 10  max_price                 98762 non-null  float64
 11  median_price              98762 non-null  float64
 12  std_price                 8594 non-null   float64
 13  total_freight_value       98762 non-null  float64
 14  avg_fr

In [17]:
# Possui linhas duplicadas pois a quantidade de order_id é menor que o número de linhas da tabela
# Porém, pode ser que um oder_id pode aparecer mais de uma vez, pois um pedido pode ter mais de uma categoria ou de produto sendo vendida
df_fct_ordem_vendedor_cat['order_id'].nunique()

97277

In [18]:
# Há pedidos com mais de uma categoria de produto sendo vendida?
(
    df_fct_ordem_vendedor_cat
    .groupby(["order_id"])
    .agg(count=("product_category_name", "nunique"))
    .reset_index()
    .sort_values(by="count", ascending=False)
)

,order_id,count
53049,8c2b13adf3f377c8f2b06b04321b0925,3
64816,ab1a70d5cd14419a6f2c2aca5ee8a79b,3
18221,3040863957c9336e7389512584639bb5,3
45061,76c4c846aae2dae9e87dfa492c3f5259,3
77433,cbb7694680a105281d391bf7002c0477,3
...,...,...
32544,55adc5cef9556477770f0b858669f8fc,1
32543,55ada6f0482ec84582a6de972bf57dd9,1
32542,55acfaef63e1cd8c8f71ab8e79b84ee0,1
32541,55ac02b6b194b6cebf53799310fb63bb,1


In [19]:
# Um pedido com mais de uma categoria de produto sendo vendida e por mais de um vendedor?
df_fct_ordem_vendedor_cat[df_fct_ordem_vendedor_cat["order_id"]=="8c2b13adf3f377c8f2b06b04321b0925"]

,order_purchase_timestamp,order_id,order_status,seller_id,product_category_name,items_sold,unique_items_sold,total_price,avg_price,min_price,max_price,median_price,std_price,total_freight_value,avg_freight_value,min_freight_value,max_freight_value,median_freight_value,std_freight_value
34414,2017-11-17 19:46:08,8c2b13adf3f377c8f2b06b04321b0925,delivered,1dfe5347016252a7884b694d4f10f5c4,cama_mesa_banho,1,1,61.00,61.00,61.00,61.00,61.00,NaN,21.08,21.08,21.08,21.08,21.08,NaN
34415,2017-11-17 19:46:08,8c2b13adf3f377c8f2b06b04321b0925,delivered,54965bbe3e4f07ae045b90b0b8541f52,cama_mesa_banho,1,1,160.00,160.00,160.00,160.00,160.00,NaN,21.08,21.08,21.08,21.08,21.08,NaN
34416,2017-11-17 19:46:08,8c2b13adf3f377c8f2b06b04321b0925,delivered,7a67c85e85bb2ce8582c35f2203ad736,cool_stuff,1,1,129.99,129.99,129.99,129.99,129.99,NaN,42.16,42.16,42.16,42.16,42.16,NaN
34417,2017-11-17 19:46:08,8c2b13adf3f377c8f2b06b04321b0925,delivered,977f9f63dd360c2a32ece2f93ad6d306,bebes,1,1,90.90,90.90,90.90,90.90,90.90,NaN,21.08,21.08,21.08,21.08,21.08,NaN


In [20]:
# Está correto. Como pode-se ver na tabela de produtos pedidos e pela coluna de ordenação do produto (order_item_id)
# Ou seja, esta tabela tem a visão de pedido na ótica do consumidor
order_items[order_items["order_id"]=="8c2b13adf3f377c8f2b06b04321b0925"]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
61467,8c2b13adf3f377c8f2b06b04321b0925,1,6f59fe49d85eb1353b826d6b5a55e753,977f9f63dd360c2a32ece2f93ad6d306,2017-11-23 20:31:40,90.90,21.08
61468,8c2b13adf3f377c8f2b06b04321b0925,2,5c818ca21204caf8ce1599617751ff49,54965bbe3e4f07ae045b90b0b8541f52,2017-11-23 20:31:40,160.00,21.08
61469,8c2b13adf3f377c8f2b06b04321b0925,3,b75ad41bddb7dc94c7e555d9f78f5e8a,1dfe5347016252a7884b694d4f10f5c4,2017-11-23 20:31:40,61.00,21.08
61470,8c2b13adf3f377c8f2b06b04321b0925,4,601a360bd2a916ecef0e88de72a6531a,7a67c85e85bb2ce8582c35f2203ad736,2017-11-23 20:31:40,129.99,42.16


In [21]:
# Também pode-se ver na tabela fato de pedidos
df_fct_ordem_pedido[df_fct_ordem_pedido["order_id"]=="8c2b13adf3f377c8f2b06b04321b0925"]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,items_sold,unique_items_sold,price,freight_value,order_value
22037,8c2b13adf3f377c8f2b06b04321b0925,0aad2e31b3c119c26acb8a47768cd00a,delivered,2017-11-17 19:46:08,2017-11-17 21:31:03,2017-11-21 12:57:04,2017-11-29 20:13:45,2017-12-20 00:00:00,4,4,441.89,105.4,547.29


In [22]:
orders[orders["order_id"]=="8c2b13adf3f377c8f2b06b04321b0925"]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
22209,8c2b13adf3f377c8f2b06b04321b0925,0aad2e31b3c119c26acb8a47768cd00a,delivered,2017-11-17 19:46:08,2017-11-17 21:31:03,2017-11-21 12:57:04,2017-11-29 20:13:45,2017-12-20 00:00:00


### Salva os dados

In [23]:
df_fct_ordem_vendedor_cat.to_csv("../data/01_interim/tb_fct_ordem_vendedor_cat.csv", index=False)